In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

In [2]:
%env CUDA_LAUNCH_BLOCKING=1

env: CUDA_LAUNCH_BLOCKING=1


In [3]:
%pip install torch tqdm numpy python-Levenshtein joblib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 75.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [4]:
data = pd.read_parquet('/kaggle/input/char-level-dataset-for-tm')
data.tail(5)

,label,feature
44421,Ọbásanjọ́ nìkan sì kọ́ ni ọ̀gágun tí ó ti jagu...,Obasanjo nikan si ko ni ogagun ti o ti jagun f...
44422,"Theophilus Danjuma, tí òun náà jẹ́ ọ̀gágun tí ...","Theophilus Danjuma, ti oun naa je ogagun ti o ..."
44423,"Ìrí-bá-kan-náà, Danjuma fi ariwo ta nípa ète b...","Iri-ba-kan-naa, Danjuma fi ariwo ta nipa ete b..."
44424,"Bákannáà ẹ̀wẹ̀wẹ̀, nínú Oṣù Kejì ọdún tí ó kọj...","Bakannaa ewewe, ninu Osu Keji odun ti o koja, ..."
44425,Ó gbàgbọ́ wípé ìgbàkanìgbàkàn “ohun tí yóò jẹ́...,O gbagbo wipe igbakanigbakan “ohun ti yoo je a...


# FEATURE ENGINEERING

In [5]:
import random

fate = random.randint(0, len(data)-1)
print(f'Printing row {fate} of the dataset.')
print('LABEL: ', data['label'][fate].strip())
print('FEATURE: ', data['feature'][fate].strip())

Printing row 41282 of the dataset.
LABEL:  Wọ́n tún mú òpó àti iṣẹ́ ọ̀tọ̀ọ̀tọ̀ — síbẹ̀síbẹ̀, àwọn ohùn, orin àti ewì wọn ti mú kí àpéròó wáyé ní orí àwọn gbàgede ìbánidọ́rẹ̀ẹ́ àwùjọ kan náà.
FEATURE:  Won tun mu opo ati ise otooto — sibesibe, awon ohun, orin ati ewi won ti mu ki aperoo waye ni ori awon gbagede ibanidoree awujo kan naa.


In [6]:
process = lambda x: [char for char in x.strip().strip()]
print('Original data: ', data['feature'][fate].strip())
print('Processed data: ', *process(data['feature'][fate].strip()))

Original data:  Won tun mu opo ati ise otooto — sibesibe, awon ohun, orin ati ewi won ti mu ki aperoo waye ni ori awon gbagede ibanidoree awujo kan naa.
Processed data:  W o n   t u n   m u   o p o   a t i   i s e   o t o o t o   —   s i b e s i b e ,   a w o n   o h u n ,   o r i n   a t i   e w i   w o n   t i   m u   k i   a p e r o o   w a y e   n i   o r i   a w o n   g b a g e d e   i b a n i d o r e e   a w u j o   k a n   n a a .


In [7]:
data.rename(columns = {'label':'diacritized', 'feature':'stripped'}, inplace=True)
data['chars_undiac'] = data['stripped'].apply(process)
data['chars_diac'] = data['diacritized'].apply(process)
data['amt_of_chars_undiac'] = data['chars_undiac'].apply(len)
data['amt_of_chars_diac'] = data['chars_diac'].apply(len)
data.tail(5)

,diacritized,stripped,chars_undiac,chars_diac,amt_of_chars_undiac,amt_of_chars_diac
44421,Ọbásanjọ́ nìkan sì kọ́ ni ọ̀gágun tí ó ti jagu...,Obasanjo nikan si ko ni ogagun ti o ti jagun f...,"[O, b, a, s, a, n, j, o, , n, i, k, a, n, , ...","[Ọ, b, á, s, a, n, j, ọ, ́, , n, ì, k, a, n, ...",74,78
44422,"Theophilus Danjuma, tí òun náà jẹ́ ọ̀gágun tí ...","Theophilus Danjuma, ti oun naa je ogagun ti o ...","[T, h, e, o, p, h, i, l, u, s, , D, a, n, j, ...","[T, h, e, o, p, h, i, l, u, s, , D, a, n, j, ...",478,495
44423,"Ìrí-bá-kan-náà, Danjuma fi ariwo ta nípa ète b...","Iri-ba-kan-naa, Danjuma fi ariwo ta nipa ete b...","[I, r, i, -, b, a, -, k, a, n, -, n, a, a, ,, ...","[Ì, r, í, -, b, á, -, k, a, n, -, n, á, à, ,, ...",117,121
44424,"Bákannáà ẹ̀wẹ̀wẹ̀, nínú Oṣù Kejì ọdún tí ó kọj...","Bakannaa ewewe, ninu Osu Keji odun ti o koja, ...","[B, a, k, a, n, n, a, a, , e, w, e, w, e, ,, ...","[B, á, k, a, n, n, á, à, , ẹ, ̀, w, ẹ, ̀, w, ...",289,299
44425,Ó gbàgbọ́ wípé ìgbàkanìgbàkàn “ohun tí yóò jẹ́...,O gbagbo wipe igbakanigbakan “ohun ti yoo je a...,"[O, , g, b, a, g, b, o, , w, i, p, e, , i, ...","[Ó, , g, b, à, g, b, ọ, ́, , w, í, p, é, , ...",271,286


In [8]:
data = data[data['amt_of_chars_diac']<=10000].drop_duplicates(subset=['diacritized'])
data = data.reset_index()
data = data[data['amt_of_chars_diac']>3]
data.sort_values(by='amt_of_chars_diac')

,index,diacritized,stripped,chars_undiac,chars_diac,amt_of_chars_undiac,amt_of_chars_diac
183,183,3...,3...,"[3, ., ., .]","[3, ., ., .]",4,4
187,187,44:.,44:.,"[4, 4, :, .]","[4, 4, :, .]",4,4
294,303,13:.,13:.,"[1, 3, :, .]","[1, 3, :, .]",4,4
345,354,Àṣe.,Ase.,"[A, s, e, .]","[À, ṣ, e, .]",4,4
362,371,Àṣẹ.,Ase.,"[A, s, e, .]","[À, ṣ, ẹ, .]",4,4
...,...,...,...,...,...,...,...
111,111,Bòyá iṣẹ lé mò ọ lọwọ 30 Bòyá òwò lò wô ọ mọl...,Boya ise le mo o lowo 30 Boya owo lo wo o mol...,"[B, o, y, a, , i, s, e, , l, e, , m, o, , ...","[B, ò, y, á, , i, ṣ, ẹ, , l, é, , m, ò, , ...",3436,3510
114,114,Bàrìnrìn àrẹ̀ bá fòni relé Táyé Ilé Kẹyìndé ni...,Barinrin are ba foni rele Taye Ile Keyinde ni ...,"[B, a, r, i, n, r, i, n, , a, r, e, , b, a, ...","[B, à, r, ì, n, r, ì, n, , à, r, ẹ, ̀, , b, ...",3461,3531
102,102,Ta lé le rí pẹ̀ léjò Pẹ̀yàn ri bímọ ri gbòrun...,Ta le le ri pe lejo Peyan ri bimo ri gborun l...,"[T, a, , l, e, , l, e, , r, i, , p, e, , ...","[T, a, , l, é, , l, e, , r, í, , p, ẹ, ̀, ...",3531,3587
110,110,O ô wulẹ̀ dìyàwò onígbáa kí-lò-n-tà 30 Má sòk...,O o wule diyawo onigbaa ki-lo-n-ta 30 Ma soko...,"[O, , o, , w, u, l, e, , d, i, y, a, w, o, ...","[O, , ô, , w, u, l, ẹ, ̀, , d, ì, y, à, w, ...",4383,4466


In [9]:
len(data)

13712

# BUILDING A VOCABULARY
* Since the `set` data type in python is a collection data type that only allows just one existence of a character in its collection. 
* We assign numbers to each item in the vocabulary (the set collection).
* During batching, while the model is training, some data may not be of the same length, this will cause complications in the training. This has been resolved by creating `<PAD>` as a unique character with a value of `0` to merely act as a filler. The model knows that `0` is just a filler and will just overlook it during batching.
* The <UNK> in the vocabulary is also there for the inference engine of the model to use for characters it cannot see.

# IMPORTANT
* We had the option of using the Unicode to represent the characters, however this would affect the learning as we learnt that the diacritized characters and some tone marked characters are represented differently from same characters created via typing on a word editor (MS. Word) in the machine, this would mean that they also posses different unicode values.
* See cell below for demonstration.

* Our decision for representing the characters with any other integer, and not the unicode value is based on the discovery as discussed above.

In [10]:
test1 = 'Àwọn' # An excerpt of the dataset to see the unicode values of the characters
test2 = 'Àwọn' # Replicated with microsoft word to see the unicode values of the characters

check = {i:ord(i) for i in test1+test2}
print(len(check))
print('Character', 'Unicode', sep = '\t')
for i in check:
    print(f'{i}\t{check[i]}')

8
Character	Unicode
À	192
w	119
ọ	7885
n	110
A	65
̀	768
o	111
̣	803


In [11]:
print('Character', 'Unicode', sep = '\t')
for i in test1:
    print(i, ord(i), sep = '\t')

Character	Unicode
À	192
w	119
ọ	7885
n	110


In [12]:
print('Character', 'Unicode', sep = '\t')
for i in test2:
    print(i, ord(i), sep = '\t')

Character	Unicode
A	65
̀	768
w	119
o	111
̣	803
n	110


We can already see that the diacritized and tone-marked values from `test1` possess a different unicode from those in `test2`.




In [13]:
letters = set()
for i in data['chars_diac'].to_list() + data['chars_undiac'].to_list():
    letters.update(list(i))

char2idx = {ch: i+1 for i, ch in enumerate(sorted(letters))}  # start from 1
char2idx['PAD'] = 0  # padding
char2idx['UNK'] = len(char2idx) - 1  # maximum valid index

idx2char = {i: ch for ch, i in char2idx.items()}
vocab_size = len(char2idx)

print(char2idx)
print(f'Vocabulary size = {vocab_size}')
print(f'Max index = {max(char2idx.values())}, PAD = {char2idx["PAD"]}, UNK = {char2idx["UNK"]}')


{'\t': 1, ' ': 2, '!': 3, '"': 4, '#': 5, '$': 6, '%': 7, '&': 8, "'": 9, '(': 10, ')': 11, '*': 12, '+': 13, ',': 14, '-': 15, '.': 16, '/': 17, '0': 18, '1': 19, '2': 20, '3': 21, '4': 22, '5': 23, '6': 24, '7': 25, '8': 26, '9': 27, ':': 28, ';': 29, '=': 30, '>': 31, '?': 32, '@': 33, 'A': 34, 'B': 35, 'C': 36, 'D': 37, 'E': 38, 'F': 39, 'G': 40, 'H': 41, 'I': 42, 'J': 43, 'K': 44, 'L': 45, 'M': 46, 'N': 47, 'O': 48, 'P': 49, 'Q': 50, 'R': 51, 'S': 52, 'T': 53, 'U': 54, 'V': 55, 'W': 56, 'X': 57, 'Y': 58, 'Z': 59, '[': 60, '\\': 61, ']': 62, '_': 63, '`': 64, 'a': 65, 'b': 66, 'c': 67, 'd': 68, 'e': 69, 'f': 70, 'g': 71, 'h': 72, 'i': 73, 'j': 74, 'k': 75, 'l': 76, 'm': 77, 'n': 78, 'o': 79, 'p': 80, 'q': 81, 'r': 82, 's': 83, 't': 84, 'u': 85, 'v': 86, 'w': 87, 'x': 88, 'y': 89, 'z': 90, '{': 91, '|': 92, '\xa0': 93, '£': 94, '§': 95, '©': 96, '«': 97, '\xad': 98, '®': 99, '¸': 100, '»': 101, 'À': 102, 'Á': 103, 'È': 104, 'É': 105, 'Ì': 106, 'Í': 107, 'Ò': 108, 'Ó': 109, 'Ô': 110,

# MODEL DEVELOPMENT
* We are splitting on 80/10/10 for training, testing and validation respectively.

In [14]:
num_epochs = 10

In [15]:
from sklearn.model_selection import train_test_split

train, temp = train_test_split(data, test_size=0.2, random_state=42) # 80% from train, the remaining goes to `temp`
valid, test = train_test_split(temp, test_size=0.5, random_state=42) # The temp is then split into half for validation and test sets.

In [16]:
# Function to encode sentences into sequences of indices using the vocabulary created above

def encode_sentence(sentence, char2idx):
    return [
        char2idx.get(ch, char2idx['UNK'])
        for ch in sentence
    ]

In [17]:
# Sequence padding function to pad dataset into uniform length batches

from torch.nn.utils.rnn import pad_sequence
import torch.nn.utils as utils
import torch

def pad_sequences(sequences, pad_value=char2idx['PAD']):
    return pad_sequence(
        [torch.tensor(seq) for seq in sequences],
        batch_first=True,
        padding_value=pad_value
    )

In [18]:
# A function to map the entire dataset into indices using the above functions and vocabulary


def prepare_data(df, char2idx):
    X = [encode_sentence(s, char2idx) for s in df['stripped']]
    y = [encode_sentence(s, char2idx) for s in df['diacritized']]
    
    max_len = max(
        max(len(seq) for seq in X),
        max(len(seq) for seq in y)
    )

    X_pad = [seq + [0] * (max_len - len(seq)) for seq in X]
    y_pad = [seq + [0] * (max_len - len(seq)) for seq in y]

    return torch.tensor(X_pad), torch.tensor(y_pad)


In [19]:
# Map the datasets into indices from vocabulary
X_train, y_train = prepare_data(train, char2idx)
X_valid, y_valid = prepare_data(valid, char2idx)
X_test,  y_test  = prepare_data(test, char2idx)

In [20]:
X_train.shape, y_train.shape, X_valid.shape, y_valid.shape, X_test.shape, y_test.shape

(torch.Size([10969, 5767]),
 torch.Size([10969, 5767]),
 torch.Size([1371, 3490]),
 torch.Size([1371, 3490]),
 torch.Size([1372, 4467]),
 torch.Size([1372, 4467]))

In [21]:
from torch.utils.data import Dataset, DataLoader

class DiacriticsDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


In [22]:
train_loader = DataLoader(
    DiacriticsDataset(X_train, y_train),
    batch_size=32,
    shuffle=True
)

valid_loader = DataLoader(
    DiacriticsDataset(X_valid, y_valid),
    batch_size=30,
    shuffle = True
)

test_loader = DataLoader(
    DiacriticsDataset(X_test, y_test),
    batch_size=15,
    shuffle=True
)

In [23]:
import torch.nn as nn

class CharBiLSTM(nn.Module):
    def __init__(self, vocab_size, emb_dim=512, hidden_dim=128):
        super().__init__()
        
        self.embedding = nn.Embedding(
            num_embeddings=vocab_size,
            embedding_dim=emb_dim,
            padding_idx=0
        )
        
        self.lstm = nn.LSTM(
            emb_dim,
            hidden_dim,
            batch_first=True,
            bidirectional=True
        )
        
        self.fc = nn.Linear(hidden_dim * 2, vocab_size)

    def forward(self, x):
        x = self.embedding(x)
        out, _ = self.lstm(x)
        out = self.fc(out)
        return out

In [24]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CharBiLSTM(vocab_size).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=0, label_smoothing = 0.05)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay = 1e-2)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer,
    T_max=num_epochs,
    eta_min = 0
)

In [25]:
def train_model(model, loader):
    model.train()
    total_loss = 0

    progress = tqdm(loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=True)
    for X, y in progress:
        X, y = X.to(device), y.to(device)
        
        optimizer.zero_grad()
        outputs = model(X)
        
        loss = criterion(
            outputs.view(-1, vocab_size),
            y.view(-1)
        )
        
        loss.backward()
        utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        
        total_loss += loss.item()
        
        progress.set_postfix(loss = f'{loss.item():.4f}')
        
    scheduler.step()
    return total_loss / len(loader)


In [26]:
import Levenshtein

def indices_to_string(indices, idx2char):
    return ''.join(
        idx2char[i] for i in indices if i != 0
    )

def cer(ref, hyp):
    return Levenshtein.distance(ref, hyp) / max(1, len(ref))

def wer(ref, hyp):
    ref_w = ref.split()
    hyp_w = hyp.split()
    return Levenshtein.distance(
        ' '.join(ref_w),
        ' '.join(hyp_w)
    ) / max(1, len(ref_w))

def sentence_acc(ref, hyp):
    return int(ref == hyp)


In [27]:
def evaluate(
    model,
    loader,
    criterion,
    idx2char,
    device
):
    model.eval()

    total_loss = 0.0
    cer_scores = []
    wer_scores = []
    lev_dists = []
    sent_accs = []

    progress = tqdm(loader, desc="Evaluating", leave=True)

    with torch.no_grad():
        for X, y in progress:
            X = X.to(device)
            y = y.to(device)

            outputs = model(X)  # (B, T, V)

            # ---- Loss (PAD masked) ----
            loss = criterion(
                outputs.view(-1, outputs.size(-1)),
                y.view(-1)
            )
            total_loss += loss.item()

            # ---- Decode predictions ----
            preds = torch.argmax(outputs, dim=-1)

            # ---- Metrics per sentence ----
            for i in range(X.size(0)):
                ref = indices_to_string(y[i].cpu().tolist(), idx2char)
                hyp = indices_to_string(preds[i].cpu().tolist(), idx2char)

                cer_scores.append(cer(ref, hyp))
                wer_scores.append(wer(ref, hyp))
                lev_dists.append(Levenshtein.distance(ref, hyp))
                sent_accs.append(sentence_acc(ref, hyp))

            progress.set_postfix(loss=f"{loss.item():.4f}")

    return {
        "loss": total_loss / len(loader),
        "CER": np.mean(cer_scores),
        "WER": np.mean(wer_scores),
        "Levenshtein": np.mean(lev_dists),
        "Sentence_Accuracy": np.mean(sent_accs)
    }


In [28]:
def predict(text, model, char2idx, idx2char):
    model.eval()
    
    encoded = encode_sentence(text, char2idx)
    tensor = torch.tensor(encoded).unsqueeze(0).to(device)
    
    with torch.no_grad():
        outputs = model(tensor)
        preds = outputs.argmax(dim=-1).squeeze(0)
    
    return ''.join(idx2char[i.item()] for i in preds)


In [29]:

for epoch in range(num_epochs):
    train_loss = train_model(model, train_loader)
    val_metrics = evaluate(
        model,
        valid_loader,
        criterion,
        idx2char,
        device
    )

    print(f"\nEpoch {epoch+1}/{num_epochs}")
    print(f"Train Loss: {train_loss:.4f}")
    print(f"Val Loss:   {val_metrics['loss']:.4f}")
    print(f"CER:        {val_metrics['CER']:.4f}")
    print(f"WER:        {val_metrics['WER']:.4f}")
    print(f"Lev Dist:   {val_metrics['Levenshtein']:.4f}")
    print(f"Sentence Acc: {val_metrics['Sentence_Accuracy']:.4f}")



Epoch 1/10:   0%|          | 0/343 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/46 [00:00<?, ?it/s]


Epoch 1/10
Train Loss: 3.1743
Val Loss:   2.8299
CER:        53.4328
WER:        5.4145
Lev Dist:   3441.0241
Sentence Acc: 0.0000


Epoch 2/10:   0%|          | 0/343 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/46 [00:00<?, ?it/s]

...

Evaluating:   0%|          | 0/46 [00:00<?, ?it/s]


Epoch 9/10
Train Loss: 2.5235
Val Loss:   2.5530
CER:        53.3639
WER:        12.1448
Lev Dist:   3434.4092
Sentence Acc: 0.0000


Epoch 10/10:   0%|          | 0/343 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/46 [00:00<?, ?it/s]


Epoch 10/10
Train Loss: 2.5171
Val Loss:   2.5444
CER:        53.3583
WER:        12.1506
Lev Dist:   3433.9081
Sentence Acc: 0.0000


In [30]:
import joblib
torch.save(model.state_dict(), 'char_bilstm_diacritizer.pt')
joblib.dump(char2idx, 'char2idx.pkl')
joblib.dump(idx2char, 'idx2char.pkl')


['idx2char.pkl']

In [31]:
model.load_state_dict(torch.load("char_bilstm_diacritizer.pt", map_location="cpu"))
char2idx = joblib.load("char2idx.pkl")
idx2char = joblib.load("idx2char.pkl")

model.eval()
model = model.to(device)


In [32]:
predict('Ekaaro, mo fe ra osan wewe ati ogede pupo.', model, char2idx, idx2char)

'Èkkàrr, mo ffẹ r   sàn wwww àtt  ggddépppp'